# Libraries

In [65]:
!nvidia-smi

Fri Apr 28 05:48:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [66]:
from google.colab import drive

drive.mount('/content/drive')
ROOT_PATH = '/content/drive/My Drive/09_research_main/lab_04'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
!pip install fastai==1.0.58
!pip install torch==1.4.0
!pip install torchvision==0.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.4.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0)
ERROR: No matching distribution found for torch==1.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torchvision==0.5.0 (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.0, 0.15.1)
ERROR: No matching distribution found for torchvision==0.5.0


In [68]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

Importing the FastAI libraries:

In [69]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         23.1.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
Bottleneck                    1.3.7
branca                        0.6.0
CacheControl                  0.12.11
cached-property               1.5.2
cach

In [70]:
# from collections.abc import Iterable

In [71]:
from fastai.vision import *
from fastai.metrics import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *

In [72]:

from datetime import datetime
from time import time

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# import warnings

import fastai
# from fastai.vision import *
from fastai.callbacks import *
# from fastai.callbacks.hooks import *
# from fastai.utils.mem import *

In [73]:
from fastai.utils.show_install import show_install; show_install()



```text
=== Software === 
python        : 3.10.11
fastai        : 1.0.58
fastprogress  : 1.0.3
torch         : 2.0.0+cu118
nvidia driver : 525.85
torch cuda    : 11.8 / is available
torch cudnn   : 8700 / is enabled

=== Hardware === 
nvidia gpus   : 1
torch devices : 1
  - gpu0      : 15360MB | Tesla T4

=== Environment === 
platform      : Linux-5.10.147+-x86_64-with-glibc2.31
distro        : #1 SMP Sat Dec 10 16:00:40 UTC 2022
conda env     : Unknown
python        : /usr/bin/python3
sys.path      : /content
/env/python
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/usr/local/lib/python3.10/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.10/dist-packages/IPython/extensions
/root/.ipython
```

Please make sure to include opening/closing ``` when you paste into forums/github to make the reports appear formatted as code sections.

Optional package(s) to enhance the diagnostics can be installed with:
pip install distro
Once installed, r

# Dataset

In [74]:
def checkpath(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [75]:
# model

mVersion = 'M6.0.0'

root_folder = os.path.abspath(os.path.join(ROOT_PATH, mVersion))

model_DIR = os.path.abspath(os.path.join(root_folder, 'model'))
checkpath(model_DIR)

In [76]:
# dataset

# dVersion = 'C2.9.3'

root_folder = os.path.abspath(os.path.join(ROOT_PATH, 'forstartdata'))

classescsv_path = os.path.join(root_folder, 'classes.csv')
# classescsv_path = os.path.join(root_folder, 'classes_all_classes.csv')

In [77]:
size = 540
bs = 8

The classes are stored in a csv so taking a look at it:

In [78]:
classescsv = pd.read_csv(classescsv_path)

classescsv.head()

,_id,_class,_name
0,0,ground,ground
1,1,frontglottis,frontglottis
2,2,vfright,vfright
3,3,vfleft,vfleft


I will now seperate out the _class since it contains the classes and turn it into a list:

In [79]:
# classes = list(classescsv['_class'])
# classes

Finally defining the data our model will run on:

In [80]:
# trainpath = os.path.join(root_folder, 'train_for_base_imgs/')
# trainpath_rgb = os.path.join(root_folder, 'train_for_base_imgs_rgb/')
# trainlabel = os.path.join(root_folder, 'train_for_base_mask/')

# trainpath_ = os.path.join(root_folder, 'train_for_step_imgs/')
# trainpath_rgb_ = os.path.join(root_folder, 'train_for_step_imgs_rgb/')
# trainlabel_ = os.path.join(root_folder, 'train_for_step_mask/')

# validpath = os.path.join(root_folder, 'valid_imgs/')
# validpath_rgb = os.path.join(root_folder, 'valid_imgs_rgb/')
# validlabel = os.path.join(root_folder, 'valid_mask/')

# testpath = os.path.join(root_folder, 'test_imgs/')
# testpath_rgb = os.path.join(root_folder, 'test_imgs_rgb/')
# testlabel = os.path.join(root_folder, 'test_mask/')

In [81]:
def transform(self, tfms:Optional[Tuple[TfmList,TfmList]]=(None,None), **kwargs):
    if not tfms: tfms=(None,None)
    assert is_listy(tfms) and len(tfms) == 2
    self.train.transform(tfms[0], **kwargs)
    self.valid.transform(tfms[1], **kwargs)
    kwargs['tfm_y'] = False # Test data has no labels
    if self.test: self.test.transform(tfms[1], **kwargs)
    return self

fastai.data_block.ItemLists.transform = transform

In [82]:
def get_mask(imgs_name):
  if str(imgs_name).split('/')[-2] == 'train_for_base_imgs' :
    return trainlabel
  else:
    return validlabel

In [83]:
# get_z = lambda x: get_mask(x) + (x.stem.replace(x.stem.split('_B')[-1], 'RRR')).replace('BRRR', 'C4.png')
# # get_z = lambda x: get_mask(x) + (x.stem.replace(x.stem.split('_B')[-1], 'RRR')).replace('BRRR', 'C12.png')

In [84]:
# NEW_DATA_PATH = os.path.join(ROOT_PATH, 'data')

# new_train_for_base_mask_DIR = os.path.join(NEW_DATA_PATH, 'train_for_base_mask')
# new_train_for_base_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'train_for_base_imgs')
# new_train_for_base_jpgs_rgb_DIR = os.path.join(NEW_DATA_PATH, 'train_for_base_imgs_rgb')

# new_train_for_step_mask_DIR = os.path.join(NEW_DATA_PATH, 'train_for_step_mask')
# new_train_for_step_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'train_for_step_imgs')
# new_train_for_step_jpgs_rgb_DIR = os.path.join(NEW_DATA_PATH, 'train_for_step_imgs_rgb')

# new_valid_mask_DIR = os.path.join(NEW_DATA_PATH, 'valid_mask')
# new_valid_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'valid_imgs')
# new_valid_jpgs_rgb_DIR = os.path.join(NEW_DATA_PATH, 'valid_imgs_rgb')

# new_test_mask_DIR = os.path.join(NEW_DATA_PATH, 'test_mask')
# new_test_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'test_imgs')
# new_test_jpgs_rgb_DIR = os.path.join(NEW_DATA_PATH, 'test_imgs_rgb')

# checkpath(new_train_for_base_mask_DIR)
# checkpath(new_train_for_base_jpgs_DIR)
# checkpath(new_train_for_base_jpgs_rgb_DIR)

# checkpath(new_train_for_step_mask_DIR)  
# checkpath(new_train_for_step_jpgs_DIR)
# checkpath(new_train_for_step_jpgs_rgb_DIR)

# checkpath(new_valid_mask_DIR)
# checkpath(new_valid_jpgs_DIR)
# checkpath(new_valid_jpgs_rgb_DIR)

# checkpath(new_test_mask_DIR)
# checkpath(new_test_jpgs_DIR)
# checkpath(new_test_jpgs_rgb_DIR)

In [92]:
data = (SegmentationItemList.from_folder(root_folder)
        # .split_by_folder(train='train_for_base_imgs', valid='valid_imgs')
        # .split_by_rand_pct(.2)
        .label_from_func(get_z, classes=classes)
        # .transform(get_transforms(), size=size, tfm_y=True)
        .databunch(bs=bs)
        # .normalize(imagenet_stats)
       )

AttributeError: ignored

In [ ]:
# data

Taking a look at an image with its mask:

In [ ]:
# data.show_batch(2, figsize=(6,6))

Defining and downloading the UNet:

# Model

In [ ]:
def cust_accuracy(input:Tensor, targs:Tensor)->Rank0Tensor:
    "Computes accuracy with `targs` when `input` is bs * n_classes."
    n = targs.shape[0]
    input = input.argmax(dim=1).view(n,-1)
    targs = targs.view(n,-1)
    return (input==targs).float().mean()

In [ ]:
# Return Jaccard index, or Intersection over Union (IoU) value
def IoU(preds:Tensor, targs:Tensor, eps:float=1e-8):
    """Computes the Jaccard loss, a.k.a the IoU loss.
    Notes: [Batch size,Num classes,Height,Width]
    Args:
        targs: a tensor of shape [B, H, W] or [B, 1, H, W].
        preds: a tensor of shape [B, C, H, W]. Corresponds to
            the raw output or logits of the model. (prediction)
        eps: added to the denominator for numerical stability.
    Returns:
        iou: the average class intersection over union value 
             for multi-class image segmentation
    """
    num_classes = preds.shape[1]
    
    # Single class segmentation?
    if num_classes == 1:
        true_1_hot = torch.eye(num_classes + 1)[targs.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        true_1_hot_f = true_1_hot[:, 0:1, :, :]
        true_1_hot_s = true_1_hot[:, 1:2, :, :]
        true_1_hot = torch.cat([true_1_hot_s, true_1_hot_f], dim=1)
        pos_prob = torch.sigmoid(preds)
        neg_prob = 1 - pos_prob
        probas = torch.cat([pos_prob, neg_prob], dim=1)
        
    # Multi-class segmentation
    else:
        # Convert target to one-hot encoding
        # true_1_hot = torch.eye(num_classes)[torch.squeeze(targs,1)]
        true_1_hot = torch.eye(num_classes)[targs.squeeze(1)]
        
        # Permute [B,H,W,C] to [B,C,H,W]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        
        # Take softmax along class dimension; all class probs add to 1 (per pixel)
        probas = F.softmax(preds, dim=1)
        
    true_1_hot = true_1_hot.type(preds.type())
    
    # Sum probabilities by class and across batch images
    dims = (0,) + tuple(range(2, targs.ndimension()))
    intersection = torch.sum(probas * true_1_hot, dims) # [class0,class1,class2,...]
    cardinality = torch.sum(probas + true_1_hot, dims)  # [class0,class1,class2,...]
    union = cardinality - intersection
    iou = (intersection / (union + eps)).mean()   # find mean of class IoU values
    return iou

In [ ]:
def dice(input:Tensor, targs:Tensor, eps:float=1e-8)->Rank0Tensor:
    input = input.clone()
    targs = targs.clone()
    n = targs.shape[0]
    input = torch.softmax(input, dim=1).argmax(dim=1)
    input = input.view(n, -1)
    targs = targs.view(n, -1)
    input[input == 0] = -999
    intersect = (input == targs).sum().float()
    union = input[input > 0].sum().float() + targs[targs > 0].sum().float()
    del input, targs
    gc.collect()
    return ((2.0 * intersect + eps) / (union + eps)).mean()

In [ ]:
def fmt_now():
    return datetime.today().strftime('%Y%m%d-%H%M%S')

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()

        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.mean()

In [ ]:
from torch import nn
import torch.nn.functional as F

class FocalLoss_C(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduction='elementwise_mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduction = reduction

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction is None:
            return F_loss
        else:
            return torch.mean(F_loss)

In [ ]:
class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)
        
    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()

In [ ]:
metrics = [dice, IoU]

callback_fns = [
    ShowGraph,
    # partial(SaveModelCallback, every='epoch', name=f"{fmt_now()}_unet_resnet34"),   
    # stop training if metric no longer improve
    # partial(EarlyStoppingCallback, monitor='IoU', min_delta=0.01, patience=2),
]

learner = unet_learner(arch = models.resnet34, metrics=metrics, wd=1e-2, callback_fns=callback_fns)
learner.model_dir = model_DIR

In [87]:
model_DIR

'/content/drive/My Drive/09_research_main/lab_04/M6.0.0/model'

In [ ]:
# model.pth

In [91]:
learner = load_learner(model_DIR + '.pth')

NotADirectoryError: ignored

In [88]:
learner = unet_learner().load(model_DIR)

TypeError: ignored

In [ ]:
# learner.loss_func = FocalLoss_C()

In [ ]:
learner.loss_func

In [ ]:
# print(learner)

In [ ]:
# learner.load(model_DIR)

In [ ]:
learner.load(model_DIR.replace(mVersion, 'M_v_6.0.0'))

In [ ]:
# learner.summary()

# Train

Training the data:

In [ ]:
# learner.lr_find()
# learner.recorder.plot(skip_end=10, suggestion=True)

In [ ]:
# learner.fit_one_cycle(40, max_lr=slice(1e-3, 1e-5))

In [ ]:
# learner.recorder.plot_losses()
# learner.recorder.plot_metrics()

Since the both the losses seem small enough, lets predict:

In [ ]:
# learner.save(model_DIR)

# Predictions

In [ ]:
# learner.show_results(figsize=(15,15))

On the left is the actual images while on the left is the predictions! They look almost identical and thereby here I conclude this notebook.

If you liked this notebook, an upvote would be of tremendous help as I trot forward with my infatuation with Deep Learning :)

# Evaluation

In [ ]:
# trainpath = os.path.join(root_folder, 'train_for_base_imgs/')
# trainpath_rgb = os.path.join(root_folder, 'train_for_base_imgs_rgb/')
# trainlabel = os.path.join(root_folder, 'train_for_base_mask/')

# trainpath_ = os.path.join(root_folder, 'train_for_step_imgs/')
# trainpath_rgb_ = os.path.join(root_folder, 'train_for_step_imgs_rgb/')
# trainlabel_ = os.path.join(root_folder, 'train_for_step_mask/')

# validpath = os.path.join(root_folder, 'valid_imgs/')
# validpath_rgb = os.path.join(root_folder, 'valid_imgs_rgb/')
# validlabel = os.path.join(root_folder, 'valid_mask/')

# testpath = os.path.join(root_folder, 'test_imgs/')
# testpath_rgb = os.path.join(root_folder, 'test_imgs_rgb/')
# testlabel = os.path.join(root_folder, 'test_mask/')

In [ ]:
import cv2
import seaborn as sn
from numpy.lib.function_base import average
from skimage.transform import resize

In [ ]:
def confusion_matrix(now_imgs_DIR, now_mask_DIR, confusion_matrix):

  _SIZE = 540

  # confusion_matrix = np.zeros((12,12))

  img = open_image(now_imgs_DIR)
  mask = cv2.imread(now_mask_DIR)
  # print(now_imgs_DIR.replace('/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_2.6.1', ''))

  # img.show(figsize=(6,6))
  # plt.show()
  # plt.imshow(mask*40)
  # plt.show()

  mask = cv2.resize(mask, (_SIZE,_SIZE))
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

  pred, pred_idx, probs = learner.predict(img)

  # plt.imshow((pred_idx.numpy()*40).reshape((_SIZE,_SIZE)))
  # plt.show()

  prediction = pred_idx.numpy()
  prediction = prediction.reshape((_SIZE,_SIZE))

  for i in range(0, mask.shape[0]):#-
    for j in range(0, mask.shape[1]):#|
      # print(prediction[i][j].item(),mask[i][j].item())
      confusion_matrix[prediction[i][j].item()][mask[i][j].item()] += 1

  # print(confusion_matrix)
  return confusion_matrix

In [ ]:
def get_confusion_matrix(imgs_dir, label_dir, CLASSES):
  LEN = len(CLASSES)
  cm = np.zeros((LEN,LEN))

  for get_to_name_path in os.listdir(imgs_dir):
    now_imgs_DIR = os.path.join(imgs_dir,  get_to_name_path)
    now_mask_DIR = os.path.join(label_dir, (get_to_name_path.replace(get_to_name_path.split('_')[-1], 'RRR')).replace('_RRR', '.png'))

    cm = confusion_matrix(now_imgs_DIR, now_mask_DIR, cm)
  return cm

In [ ]:
def print_cm(GET_CM, CLASSES):

  df_cm = pd.DataFrame(GET_CM, index = [i for i in CLASSES], columns = [i for i in CLASSES])
  plt.figure(figsize = (15,12))
  sn.heatmap(df_cm, annot=True)

In [ ]:
def IoU_computer(a, CLASSES):
  LEN = len(CLASSES)
  IoUstr = []
  IoU = []

  for i in range(1, LEN):
    temp = 0
    for j in range(1, LEN):
      temp += a[j][i] + a[i][j]
    IoUstr += [str(a[i][i])+'/'+str(temp-a[i][i])]
    IoU += [a[i][i]/(temp-a[i][i])]

  ALL_IoU = average(IoU)

  print(IoUstr)
  print(IoU)
  print(f'IoU = {ALL_IoU:.2f}')

In [ ]:
def evaluation(imgs_dir, label_dir, classes):
  cm = get_confusion_matrix(imgs_dir, label_dir, classes)

  acm = cm
  acm[0][0] = 50

  print_cm(cm, classes)
  IoU_computer(cm, classes)

In [ ]:
# maim step test for acc & cm

# evaluation(trainpath_, trainlabel_, classes)

In [ ]:
# maim valid test for acc & cm

# evaluation(validpath, validlabel, classes)

In [ ]:
# maim test test for acc & cm

# evaluation(testpath, testlabel, classes)

#Export step dataset

In [ ]:
# Export dataset version

Version = 'data/dataset_for_step_' + mVersion + '_' + dVersion + '_D1.0.11'

bindingset_dir = os.path.abspath(os.path.join(ROOT_PATH, Version))

step_train_bindingset_dir =             os.path.join(bindingset_dir, 'step_train/step_train')
step_train_focused_bindingset_dir =     os.path.join(bindingset_dir, 'step_train/step_train_focused')
step_train_rgb_bindingset_dir =         os.path.join(bindingset_dir, 'step_train/step_train_rgb')
step_train_rgb_focused_bindingset_dir = os.path.join(bindingset_dir, 'step_train/step_train_rgb_focused')
step_train_pred_idx_bindingset_dir =    os.path.join(bindingset_dir, 'step_train/step_train_pred_idx')
step_train_tensor_bindingset_dir =      os.path.join(bindingset_dir, 'step_train/step_train_tensor')

step_valid_bindingset_dir =             os.path.join(bindingset_dir, 'step_valid/step_valid')
step_valid_focused_bindingset_dir =     os.path.join(bindingset_dir, 'step_valid/step_valid_focused')
step_valid_rgb_bindingset_dir =         os.path.join(bindingset_dir, 'step_valid/step_valid_rgb')
step_valid_rgb_focused_bindingset_dir = os.path.join(bindingset_dir, 'step_valid/step_valid_rgb_focused')
step_valid_pred_idx_bindingset_dir =    os.path.join(bindingset_dir, 'step_valid/step_valid_pred_idx')
step_valid_tensor_bindingset_dir =      os.path.join(bindingset_dir, 'step_valid/step_valid_tensor')

valid_bindingset_dir =                  os.path.join(bindingset_dir, 'valid/valid')
valid_focused_bindingset_dir =          os.path.join(bindingset_dir, 'valid/valid_focused')
valid_rgb_bindingset_dir =              os.path.join(bindingset_dir, 'valid/valid_rgb')
valid_rgb_focused_bindingset_dir =      os.path.join(bindingset_dir, 'valid/valid_rgb_focused')
valid_pred_idx_bindingset_dir =         os.path.join(bindingset_dir, 'valid/valid_pred_idx')
valid_tensor_bindingset_dir =           os.path.join(bindingset_dir, 'valid/valid_tensor')

test_bindingset_dir =                   os.path.join(bindingset_dir, 'test/test')
test_focused_bindingset_dir =           os.path.join(bindingset_dir, 'test/test_focused')
test_rgb_bindingset_dir =               os.path.join(bindingset_dir, 'test/test_rgb')
test_rgb_focused_bindingset_dir =       os.path.join(bindingset_dir, 'test/test_rgb_focused')
test_pred_idx_bindingset_dir =          os.path.join(bindingset_dir, 'test/test_pred_idx')
test_tensor_bindingset_dir =            os.path.join(bindingset_dir, 'test/test_tensor')


checkpath(step_train_bindingset_dir)
checkpath(step_train_focused_bindingset_dir)
checkpath(step_train_rgb_bindingset_dir)
checkpath(step_train_rgb_focused_bindingset_dir)
checkpath(step_train_pred_idx_bindingset_dir)
checkpath(step_train_tensor_bindingset_dir)

checkpath(step_valid_bindingset_dir)
checkpath(step_valid_focused_bindingset_dir)
checkpath(step_valid_rgb_bindingset_dir)
checkpath(step_valid_rgb_focused_bindingset_dir)
checkpath(step_valid_pred_idx_bindingset_dir)
checkpath(step_valid_tensor_bindingset_dir)

checkpath(valid_bindingset_dir)
checkpath(valid_focused_bindingset_dir)
checkpath(valid_rgb_bindingset_dir)
checkpath(valid_rgb_focused_bindingset_dir)
checkpath(valid_pred_idx_bindingset_dir)
checkpath(valid_tensor_bindingset_dir)

checkpath(test_bindingset_dir)
checkpath(test_focused_bindingset_dir)
checkpath(test_rgb_bindingset_dir)
checkpath(test_rgb_focused_bindingset_dir)
checkpath(test_pred_idx_bindingset_dir)
checkpath(test_tensor_bindingset_dir)

##Export non-tensor

In [ ]:
def mask_cutter(img_DIR):
  img = cv2.imread(img_DIR)
  pred_idx = learner.predict(open_image(img_DIR))[1].numpy()

  mask_ = resize(pred_idx.reshape((540, 540)), (1080, 1080))
  # print(mask_[0][0])

  for i in range(0, mask_.shape[0]):
    for j in range(0, mask_.shape[1]):
      if mask_[i][j] <= 5.5e-20:#5.421010862427522e-20
        img[i][j][0] = 255
        img[i][j][1] = 255
        img[i][j][2] = 255

  # img = resize(img, (540, 540))
  # plt.figure()
  # plt.imshow(img)
  # plt.show()

  return img

In [ ]:
def fixfix(x,y,w,h):
  if w >= h:
    y = int(y - (h / 4))
    if y < 0 : y = 0
    h = w
    if y + h > 1080 : y = 1080 - h
  else:
    x = int(x - (w / 4))
    if x < 0 : x = 0
    w = h
    if x + w > 1080 : x = 1080 - w
  return x,y,w,h

In [ ]:
def mask_cutter_v_2(img_DIR):
  img = cv2.imread(img_DIR)
  pred_idx = learner.predict(open_image(img_DIR))[1].numpy()

  mask_ = resize(pred_idx.reshape((540, 540)), (1080, 1080))

  for i in range(0, mask_.shape[0]):
    for j in range(0, mask_.shape[1]):
      if mask_[i][j] <= 5.5e-20:
        img[i][j][0] = 255
        img[i][j][1] = 255
        img[i][j][2] = 255
  
  original = img.copy()

  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

  max = 0
  cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts = cnts[0] if len(cnts) == 2 else cnts[1]
  # print(cnts)
  for c in cnts:
      x,y,w,h = cv2.boundingRect(c)
      if h > max : 
        max = h
        max_index = c

  new_img = resize(original, (540, 540))
  try:
    x,y,w,h = cv2.boundingRect(max_index)
  except UnboundLocalError:
    print('Jumping over ' + img_DIR)
    return original

  x,y,w,h = fixfix(x,y,w,h)

  # cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,255), 2)
  new_img = resize(original[y:y+h, x:x+w], (540, 540))

  new_img = new_img*255

  # plt.figure()
  # plt.imshow(new_img)
  # plt.show()

  # print(img_DIR.split('/')[-1], np.unique(new_img), x, y, w, h)

  return new_img

In [ ]:
# # run & save bindingset_v_2_trainmaker

# kokocounter = 0

# for imgs_name in os.listdir(trainpath_):
#   print(imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png', kokocounter)

#   if kokocounter%10 == 0:
#     cv2.imwrite(os.path.join(step_valid_bindingset_dir,             imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png'), mask_cutter(    os.path.join(trainpath_,     imgs_name)))
#     cv2.imwrite(os.path.join(step_valid_focused_bindingset_dir,     imgs_name.split('.')[0] + '_' + mVersion + '_gs_CFid' +  str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(trainpath_,     imgs_name)))
#     if imgs_name.split('_B')[1] == 'n00.png':
#       cv2.imwrite(os.path.join(step_valid_rgb_bindingset_dir,         imgs_name.split('.')[0] + '_' + mVersion + '_rgb_COid' + str(kokocounter) + '.png'), mask_cutter(    os.path.join(trainpath_rgb_, imgs_name.split('_B')[0] + '_Bnrgb.png')))
#       cv2.imwrite(os.path.join(step_valid_rgb_focused_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_rgb_CFid' + str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(trainpath_rgb_, imgs_name.split('_B')[0] + '_Bnrgb.png')))
#       np.save(os.path.join(step_valid_pred_idx_bindingset_dir,        imgs_name.split('.')[0] + '_' + mVersion + '_pm_COid' +  str(kokocounter)), learner.predict(open_image(os.path.join(trainpath_, imgs_name)))[1].numpy().reshape((540, 540)))
#   else:
#     cv2.imwrite(os.path.join(step_train_bindingset_dir,             imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png'), mask_cutter(    os.path.join(trainpath_,     imgs_name)))
#     cv2.imwrite(os.path.join(step_train_focused_bindingset_dir,     imgs_name.split('.')[0] + '_' + mVersion + '_gs_CFid' +  str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(trainpath_,     imgs_name)))
#     if imgs_name.split('_B')[1] == 'n00.png':
#       cv2.imwrite(os.path.join(step_train_rgb_bindingset_dir,         imgs_name.split('.')[0] + '_' + mVersion + '_rgb_COid' + str(kokocounter) + '.png'), mask_cutter(    os.path.join(trainpath_rgb_, imgs_name.split('_B')[0] + '_Bnrgb.png')))
#       cv2.imwrite(os.path.join(step_train_rgb_focused_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_rgb_CFid' + str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(trainpath_rgb_, imgs_name.split('_B')[0] + '_Bnrgb.png')))
#       np.save(os.path.join(step_train_pred_idx_bindingset_dir,        imgs_name.split('.')[0] + '_' + mVersion + '_pm_COid' +  str(kokocounter)), learner.predict(open_image(os.path.join(trainpath_, imgs_name)))[1].numpy().reshape((540, 540)))

#   kokocounter += 1
  
# print('\n\nSuccessfully Completed!!!\n\n')#797

In [ ]:
# run & save bindingset_v_2_trainmaker

kokocounter = 0

for imgs_name in os.listdir(trainpath_):
  print(imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png', kokocounter)

  cv2.imwrite(os.path.join(step_train_bindingset_dir,             imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png'), mask_cutter(    os.path.join(trainpath_,     imgs_name)))
  cv2.imwrite(os.path.join(step_train_focused_bindingset_dir,     imgs_name.split('.')[0] + '_' + mVersion + '_gs_CFid' +  str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(trainpath_,     imgs_name)))
  if imgs_name.split('_B')[1] == 'n00.png':
    cv2.imwrite(os.path.join(step_train_rgb_bindingset_dir,         imgs_name.split('.')[0] + '_' + mVersion + '_rgb_COid' + str(kokocounter) + '.png'), mask_cutter(    os.path.join(trainpath_rgb_, imgs_name.split('_B')[0] + '_Bnrgb.png')))
    cv2.imwrite(os.path.join(step_train_rgb_focused_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_rgb_CFid' + str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(trainpath_rgb_, imgs_name.split('_B')[0] + '_Bnrgb.png')))
    np.save(os.path.join(step_train_pred_idx_bindingset_dir,        imgs_name.split('.')[0] + '_' + mVersion + '_pm_COid' +  str(kokocounter)), learner.predict(open_image(os.path.join(trainpath_, imgs_name)))[1].numpy().reshape((540, 540)))

  kokocounter += 1
  
print('\n\nSuccessfully Completed!!!\n\n')#797

In [ ]:
# run & save bindingset_v_2_validmaker

kokocounter = 0

for imgs_name in os.listdir(validpath):
  print(imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png', kokocounter)

  cv2.imwrite(os.path.join(valid_bindingset_dir,             imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png'), mask_cutter(    os.path.join(validpath,     imgs_name)))
  cv2.imwrite(os.path.join(valid_focused_bindingset_dir,     imgs_name.split('.')[0] + '_' + mVersion + '_gs_CFid' +  str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(validpath,     imgs_name)))
  if imgs_name.split('_B')[1] == 'n00.png':
    cv2.imwrite(os.path.join(valid_rgb_bindingset_dir,         imgs_name.split('.')[0] + '_' + mVersion + '_rgb_COid' + str(kokocounter) + '.png'), mask_cutter(    os.path.join(validpath_rgb, imgs_name.split('_B')[0] + '_Bnrgb.png')))
    cv2.imwrite(os.path.join(valid_rgb_focused_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_rgb_CFid' + str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(validpath_rgb, imgs_name.split('_B')[0] + '_Bnrgb.png')))
    np.save(os.path.join(valid_pred_idx_bindingset_dir,        imgs_name.split('.')[0] + '_' + mVersion + '_pm_COid' +  str(kokocounter)), learner.predict(open_image(os.path.join(validpath, imgs_name)))[1].numpy().reshape((540, 540)))

  kokocounter += 1
  
print('\n\nSuccessfully Completed!!!\n\n')#797

In [ ]:
# run & save bindingset_v_2_testmaker

kokocounter = 0

for imgs_name in os.listdir(testpath):
  print(imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png', kokocounter)

  cv2.imwrite(os.path.join(test_bindingset_dir,             imgs_name.split('.')[0] + '_' + mVersion + '_gs_COid' +  str(kokocounter) + '.png'), mask_cutter(    os.path.join(testpath,     imgs_name)))
  cv2.imwrite(os.path.join(test_focused_bindingset_dir,     imgs_name.split('.')[0] + '_' + mVersion + '_gs_CFid' +  str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(testpath,     imgs_name)))
  if imgs_name.split('_B')[1] == 'n00.png':
    cv2.imwrite(os.path.join(test_rgb_bindingset_dir,         imgs_name.split('.')[0] + '_' + mVersion + '_rgb_COid' + str(kokocounter) + '.png'), mask_cutter(    os.path.join(testpath_rgb, imgs_name.split('_B')[0] + '_Bnrgb.png')))
    cv2.imwrite(os.path.join(test_rgb_focused_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_rgb_CFid' + str(kokocounter) + '.png'), mask_cutter_v_2(os.path.join(testpath_rgb, imgs_name.split('_B')[0] + '_Bnrgb.png')))
    np.save(os.path.join(test_pred_idx_bindingset_dir,        imgs_name.split('.')[0] + '_' + mVersion + '_pm_COid' +  str(kokocounter)), learner.predict(open_image(os.path.join(testpath, imgs_name)))[1].numpy().reshape((540, 540)))
  kokocounter += 1
  
print('\n\nSuccessfully Completed!!!\n\n')#797

In [ ]:
# END HERE np.load('my_array.npy')

##Export tensor

In [ ]:
# # run & save bindingset_v_3_trainmaker

# kokocounter = 0

# for imgs_name in os.listdir(trainpath_):
#   print(imgs_name.split('.')[0] + '_' + mVersion + '_tensor_COid' +  str(kokocounter) + '.pt', kokocounter)

#   if kokocounter%10 == 0:
#     torch.save(learner.predict(open_image(os.path.join(trainpath_, imgs_name)))[2], os.path.join(step_valid_tensor_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_tensor_COid' +  str(kokocounter) + '.pt'))
#   else:
#     torch.save(learner.predict(open_image(os.path.join(trainpath_, imgs_name)))[2], os.path.join(step_train_tensor_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_tensor_COid' +  str(kokocounter) + '.pt'))

#   kokocounter += 1
  
# print('\n\nSuccessfully Completed!!!\n\n')#797

In [ ]:
# # run & save bindingset_v_3_validmaker

# kokocounter = 0

# for imgs_name in os.listdir(trainpath_):
#   print(imgs_name.split('.')[0] + '_' + mVersion + '_tensor_COid' +  str(kokocounter) + '.pt', kokocounter)

#   torch.save(learner.predict(open_image(os.path.join(validpath, imgs_name)))[2], os.path.join(valid_tensor_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_tensor_COid' +  str(kokocounter) + '.pt'))

#   kokocounter += 1
  
# print('\n\nSuccessfully Completed!!!\n\n')#797

In [ ]:
# # run & save bindingset_v_3_testmaker

# kokocounter = 0

# for imgs_name in os.listdir(trainpath_):
#   print(imgs_name.split('.')[0] + '_' + mVersion + '_tensor_COid' +  str(kokocounter) + '.pt', kokocounter)

#   torch.save(learner.predict(open_image(os.path.join(testpath, imgs_name)))[2], os.path.join(test_tensor_bindingset_dir, imgs_name.split('.')[0] + '_' + mVersion + '_tensor_COid' +  str(kokocounter) + '.pt'))

#   kokocounter += 1
  
# print('\n\nSuccessfully Completed!!!\n\n')#797

In [ ]:
# END HERE